Our Sources of data are as follows:


1st dataset: https://data.world/ https://data.world/noaa/severe-weather-data-hail-2017

2nd dqtaset: https://nauticalcharts.noaa.gov/data/us-maritime-limits-and-boundaries.html

In [1]:
#import the requirements
import numpy as np
import pandas as pd
import datetime
import pyodbc
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()


In [2]:
# Import data sources 
hail2017 = "hail2017r.xlsx"
longlat  = "longlat.csv"

In [3]:
# The correct encoding must be used to read the CSV in pandas
dfgeo = pd.read_csv(longlat, encoding="ISO-8859-1")
dfhail2017 = pd.read_excel(hail2017, encoding="ISO-8859-1")

In [4]:
dfhail2017.head()

,ztime,longitude,latitude,wsrid,cellid,range,azimuth,sevprob,prob,maxsize
0,20180201000156,129.34744,25.74558,PODN,Z5,85,113,0,30,0.5
1,20180201000156,128.70164,24.87586,PODN,Y4,96,153,40,80,0.5
2,20180201004849,-159.24291,21.93735,PHMO,C3,125,293,-999,-999,-999.0
3,20180201034401,-158.72963,22.77956,PHMO,W4,131,319,-999,-999,-999.0
4,20180202180626,-161.12008,20.40804,PHKI,E2,125,225,-999,-999,-999.0


Data Cleaning 
(cleaning, joining, filtering, aggregating, etc).

In [11]:
#convert to date
dfhail2017['ztime']=pd.to_datetime(dfhail2017['ztime'],format='%Y%m%d%H%M%S')

In [ ]:
#df['DATE'] = pd.to_datetime(df['DATE'],format='%d-%b-%y %H:%M')

In [ ]:
#dfhail2017['ztime']=pd.to_datetime(dfhail2017['ztime'],format='%d-%b-%y %H:%M')

In [12]:
# Delete extraneous column
dfhail2017.drop(['azimuth','range','cellid','azimuth'], axis=1, inplace=True)
dfhail2017.head()

,ztime,longitude,latitude,wsrid,sevprob,prob,maxsize
0,2018-02-01 00:01:56,129.34744,25.74558,PODN,0,30,0.5
1,2018-02-01 00:01:56,128.70164,24.87586,PODN,40,80,0.5
2,2018-02-01 00:48:49,-159.24291,21.93735,PHMO,-999,-999,-999.0
3,2018-02-01 03:44:01,-158.72963,22.77956,PHMO,-999,-999,-999.0
4,2018-02-02 18:06:26,-161.12008,20.40804,PHKI,-999,-999,-999.0


In [13]:
dfhail2017.ztime.unique()

array(['2018-02-01T00:01:56.000000000', '2018-02-01T00:48:49.000000000',
       '2018-02-01T03:44:01.000000000', ...,
       '2018-02-02T18:00:25.000000000', '2018-02-02T18:05:51.000000000',
       '2018-02-06T23:56:12.000000000'], dtype='datetime64[ns]')

In [14]:
dfhail2017.tail()

,ztime,longitude,latitude,wsrid,sevprob,prob,maxsize
82017,2018-02-06 23:55:24,-96.70565,32.31378,KDAL,0,20,0.5
82018,2018-02-06 23:55:24,-96.87693,32.29618,KDAL,0,20,0.5
82019,2018-02-06 23:55:24,-96.74091,33.17915,KDAL,50,90,0.5
82020,2018-02-06 23:56:12,-95.77747,32.33720,KGRK,-999,-999,-999.0
82021,2018-02-06 23:56:32,-155.34243,19.57674,PHKM,0,90,0.5


In [15]:
dfhail2017['ztime'].value_counts()

2018-02-20 13:44:23    44
2018-02-20 11:02:48    40
2018-02-20 13:29:18    34
2018-02-20 13:38:24    34
2018-02-06 23:19:27    34
2018-02-20 13:41:16    31
2018-02-20 10:59:42    30
2018-02-06 23:16:36    30
2018-02-06 23:10:37    30
2018-02-20 13:14:27    29
2018-02-20 12:56:30    28
2018-02-06 23:28:35    28
2018-02-20 13:53:14    27
2018-02-20 12:53:24    27
2018-02-20 12:50:31    27
2018-02-20 13:47:15    27
2018-02-07 00:13:22    27
2018-02-06 23:31:26    26
2018-02-20 13:45:59    26
2018-02-20 13:02:29    26
2018-02-20 13:23:18    26
2018-02-20 11:50:41    26
2018-02-07 00:04:32    26
2018-02-20 13:26:25    26
2018-02-06 23:52:33    25
2018-02-20 13:35:17    25
2018-02-20 13:08:28    25
2018-02-21 06:11:21    25
2018-02-20 13:20:26    24
2018-02-06 23:43:25    24
                       ..
2018-02-14 17:14:19     1
2018-02-18 20:32:42     1
2018-02-10 10:58:43     1
2018-02-10 07:26:43     1
2018-02-14 17:42:22     1
2018-02-05 05:41:02     1
2018-02-13 16:28:24     1
2018-02-12 2

In [16]:
#adding breakout of date for further analysis
dfhail2017['year'] = pd.DatetimeIndex(dfhail2017['ztime']).year
dfhail2017['quarter'] = pd.DatetimeIndex(dfhail2017['ztime']).quarter
dfhail2017['month'] = pd.DatetimeIndex(dfhail2017['ztime']).month
dfhail2017['weekofyear'] = pd.DatetimeIndex(dfhail2017['ztime']).weekofyear
dfhail2017.head()

,ztime,longitude,latitude,wsrid,sevprob,prob,maxsize,year,quarter,month,weekofyear
0,2018-02-01 00:01:56,129.34744,25.74558,PODN,0,30,0.5,2018,1,2,5
1,2018-02-01 00:01:56,128.70164,24.87586,PODN,40,80,0.5,2018,1,2,5
2,2018-02-01 00:48:49,-159.24291,21.93735,PHMO,-999,-999,-999.0,2018,1,2,5
3,2018-02-01 03:44:01,-158.72963,22.77956,PHMO,-999,-999,-999.0,2018,1,2,5
4,2018-02-02 18:06:26,-161.12008,20.40804,PHKI,-999,-999,-999.0,2018,1,2,5


In [17]:
new_df = pd.merge(dfgeo, dfhail2017, on=['wsrid'])
new_df.head()

,wsrid,city,statename,ztime,longitude,latitude,sevprob,prob,maxsize,year,quarter,month,weekofyear
0,KFTG,FRONT RANGE AP,CO,2018-02-03 22:34:52,-103.78934,40.74133,0,80,0.50,2018,1,2,5
1,KFTG,FRONT RANGE AP,CO,2018-02-03 22:38:56,-103.73400,40.73428,0,50,0.50,2018,1,2,5
2,KFTG,FRONT RANGE AP,CO,2018-02-03 22:43:00,-103.67011,40.70019,0,20,0.50,2018,1,2,5
3,KFTG,FRONT RANGE AP,CO,2018-02-03 22:51:33,-103.55471,40.65027,0,10,0.50,2018,1,2,5
4,KDMX,JOHNSTON,IA,2018-02-15 09:35:49,-91.37036,42.59278,50,80,0.75,2018,1,2,7


In [18]:
new_df1 = new_df.groupby('statename')['wsrid'].count()
new_df1.head()

statename
 AK      23
 AL    8219
 AR     786
 AZ      92
 CA     144
Name: wsrid, dtype: int64

In [19]:
rds_connection_string = "user:password@127.0.0.1/hail_db"
engine = create_engine(f'mysql://{rds_connection_string}')


In [24]:
#loading the data from dataframe to sql
new_df.to_sql(name='geolocation', con=engine, if_exists='append', index=False)

In [25]:
#confirm that your data has been loaded to sql

pd.read_sql_query('select * from geolocation', con=engine).head()

,wsrid,city,statename,ztime,longitude,latitude,sevprob,prob,maxsize,year,quarter,month,weekofyear
0,KFTG,FRONT RANGE AP,CO,2018-02-03 22:34:52,-103.78934,40.74133,0,80,0.50,2018,1,2,5
1,KFTG,FRONT RANGE AP,CO,2018-02-03 22:38:56,-103.73400,40.73428,0,50,0.50,2018,1,2,5
2,KFTG,FRONT RANGE AP,CO,2018-02-03 22:43:00,-103.67011,40.70019,0,20,0.50,2018,1,2,5
3,KFTG,FRONT RANGE AP,CO,2018-02-03 22:51:33,-103.55471,40.65027,0,10,0.50,2018,1,2,5
4,KDMX,JOHNSTON,IA,2018-02-15 09:35:49,-91.37036,42.59278,50,80,0.75,2018,1,2,7


In [26]:
#confirm data by querying sql
pd.read_sql_query('select * from geolocation', con=engine).head()

,wsrid,city,statename,ztime,longitude,latitude,sevprob,prob,maxsize,year,quarter,month,weekofyear
0,KFTG,FRONT RANGE AP,CO,2018-02-03 22:34:52,-103.78934,40.74133,0,80,0.50,2018,1,2,5
1,KFTG,FRONT RANGE AP,CO,2018-02-03 22:38:56,-103.73400,40.73428,0,50,0.50,2018,1,2,5
2,KFTG,FRONT RANGE AP,CO,2018-02-03 22:43:00,-103.67011,40.70019,0,20,0.50,2018,1,2,5
3,KFTG,FRONT RANGE AP,CO,2018-02-03 22:51:33,-103.55471,40.65027,0,10,0.50,2018,1,2,5
4,KDMX,JOHNSTON,IA,2018-02-15 09:35:49,-91.37036,42.59278,50,80,0.75,2018,1,2,7
